<a href="https://colab.research.google.com/github/jylee2930/Basic_BIgDataAnalysis/blob/main/Temp_Analysis(Random_Forest).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import datetime, timedelta

In [ ]:
tmp = pd.read_csv("/content/wonju_temp.csv")

##날짜 데이터를 time_series 형식으로 변환
tmp['date'] = pd.to_datetime(tmp['date'], errors='coerce')
tmp['day_of_week'] = tmp['date'].dt.dayofweek
tmp['month'] = tmp['date'].dt.month
tmp['year'] = tmp['date'].dt.year
##onehot encoding, 요일별로 컬럼이 생성되며, 머신러닝에서 요일을 범주형으로 활용
tmp['day_of_week'] = tmp['day_of_week'].astype('category')
tmp = pd.get_dummies(tmp, columns=['day_of_week'], prefix='w', drop_first=True, dtype=int)
print(tmp.head(10))


In [ ]:
fig,axes = plt.subplots(4,1, figsize = (25,20))
axes[0].plot(tmp['date'], tmp['mean_tmp'], label = 'mean_tmp')
axes[0].set_title("Mean_Teampreture")

axes[1].plot(tmp['date'], tmp['min_tmp'],label = 'min_tmp')
axes[1].set_title("Min_Teampreture")

axes[2].plot(tmp['date'], tmp['max_tmp'],label = 'max_tmp')
axes[2].set_title("Max_Teampreture")

axes[3].plot(tmp['date'], tmp['mean_tmp'],label = 'mean_tmp')
axes[3].plot(tmp['date'], tmp['min_tmp'],label = 'min_tmp')
axes[3].plot(tmp['date'], tmp['max_tmp'],label = 'max_tmp')
axes[3].set_title("Teampreture")
plt.legend()
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
##평균기온 예측, 평균기온이 타겟데이터 y
# 1. 피처 엔지니어링 - 날짜에서 유용한 특성 추출
tmp['season'] = tmp['month'].map({12: 'Winter', 1: 'Winter', 2: 'Winter',
                                3: 'Spring', 4: 'Spring', 5: 'Spring',
                                6: 'Summer', 7: 'Summer', 8: 'Summer',
                                9: 'Autumn', 10: 'Autumn', 11: 'Autumn'})

# 계절을 숫자로 인코딩
season_encoder = LabelEncoder()
tmp['season_encoded'] = season_encoder.fit_transform(tmp['season'])
print(tmp.head())

In [ ]:
# 2. 피처와 타겟 변수 설정
# 타겟 변수: mean_tmp (평균기온)
# 피처 변수: area, 날짜, 요일
# 타겟 변수: mean_tmp (평균기온)
# 피처 변수: area, month, year, 요일 더미변수들 (min_tmp, max_tmp 제외)
feature_columns = ['area', 'month', 'year', 'w_1.0', 'w_2.0', 'w_3.0', 'w_4.0', 'w_5.0', 'w_6.0']
X = tmp[feature_columns]
y = tmp['mean_tmp']
print(X[:10])
print(y[:10])

In [ ]:
##3. 학습 데이터와 검증 데이터로 구분
from sklearn.model_selection import train_test_split

# Drop rows with NaN in the target variable
tmp_cleaned = tmp.dropna(subset=['mean_tmp'])

# Define features and target after dropping NaNs
feature_columns = ['area', 'month', 'year', 'w_1.0', 'w_2.0', 'w_3.0', 'w_4.0', 'w_5.0', 'w_6.0']
X = tmp_cleaned[feature_columns]
y = tmp_cleaned['mean_tmp']

# 방법 2: 데이터 분할 (7:3 비율)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True
)
print(len(X_train))
print(len(y_train))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
##4. Random Forest 모델 생성 및 학습
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
#5. 예측
y_pred = rf_model.predict(X_test)

#6. 성능 평가
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# 7. 모델 결과 저장
best_model = rf_model
best_predictions = y_pred
best_model_name = 'Random Forest'

In [ ]:
# 8. 예측 결과 시각화
plt.figure(figsize=(12, 5))

# 실제값 vs 예측값 산점도
plt.subplot(1, 2, 1)
plt.scatter(y_test, best_predictions, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Tempreture')
plt.ylabel('Predict-tempreture')
plt.title('Tempreture vs Preict(Random Forest)')

# 잔차 플롯
plt.subplot(1, 2, 2)
residuals = y_test - best_predictions
plt.scatter(best_predictions, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predict')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.tight_layout()
plt.show()

In [ ]:
print(X_test[:10])

In [ ]:
##예측 데이터와 비교 그래프 그리기
## 원본 인덱스를 이용해 날짜 정보 복원
test_indices = X_test.index
test_dates = tmp.loc[test_indices, 'date']

# 날짜순으로 정렬을 위한 데이터프레임 생성
comparison_tmp = pd.DataFrame({
    'date': test_dates,
    'year': X_test['year'],
    'month': X_test['month'],
    'actual': y_test,
    'predicted': best_predictions
}).sort_values('date').reset_index(drop=True)
residuals =comparison_tmp['actual']-comparison_tmp['predicted']

plt.figure(figsize=(15, 10))
plt.scatter(comparison_tmp['year'], comparison_tmp['actual'],
           alpha=0.6, label='Actual Temperature', s=30, color='blue')
plt.scatter(comparison_tmp['year'], comparison_tmp['predicted'],
           alpha=0.6, label='Predicted Temperature', s=30, color='red')
plt.xlabel('Year')
plt.ylabel('Temperature (°C)')
plt.title('Comparison: Predicted vs Temperature by Year')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()



In [ ]:
#시계열 순서대로 비교 (추천)
plt.figure(figsize=(20, 8))
plt.plot(comparison_tmp['date'], comparison_tmp['actual'],
         label='Actual Temperature', linewidth=2, alpha=0.8)
plt.plot(comparison_tmp['date'], comparison_tmp['predicted'],
         label='Predicted Temperature', linewidth=2, alpha=0.8, linestyle='--')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.title('Time Series: Predicted vs Actual Temperature')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.scatter(comparison_tmp['date'], residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Date(Year)')
plt.ylabel('Residuals')
plt.title('Residual Plot by Year')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(comparison_tmp['date'], residuals, label='residuals', linewidth=1.5, alpha=0.8)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Date(Year)')
plt.ylabel('Residuals')
plt.title('Residual Plot by Year')
plt.legend()
plt.show()